In [1]:
import matplotlib.pyplot as plt
import datetime
#from sshtunnel import SSHTunnelForwarder
#import psycopg2
from pandas import read_csv

In [2]:
#Read CSV File
master = read_csv('~/bridge/sept01.csv')
print(master.head())
accel_1_x = master['accel_1_x'][:]
accel_1_y = master['accel_1_y'][:]
accel_1_z = master['accel_1_z'][:]

accel_2_x = master['accel_2_x'][:]
accel_2_y = master['accel_2_y'][:]
accel_2_z = master['accel_2_z'][:]

accel_3_x = master['accel_3_x'][:]
accel_3_y = master['accel_3_y'][:]
accel_3_z = master['accel_3_z'][:]

accel_4_x = master['accel_4_x'][:]
accel_4_y = master['accel_4_y'][:]
accel_4_z = master['accel_4_z'][:]

accel_5_x = master['accel_5_x'][:]
accel_5_y = master['accel_5_y'][:]
accel_5_z = master['accel_5_z'][:]

accel_6_x = master['accel_6_x'][:]
accel_6_y = master['accel_6_y'][:]
accel_6_z = master['accel_6_z'][:]

received = master['received'][:]
print(accel_1_x.head())

          id     eventtime  accel_1_x  accel_1_y  accel_1_z  accel_2_x  \
0  129881743  1.567282e+09   6.079945   5.970705   3.352367   5.950346   
1  129881744  1.567282e+09   6.078339   5.977462   3.354489   5.942620   
2  129881745  1.567282e+09   6.083371   5.972439   3.351835   5.947178   
3  129881746  1.567282e+09   6.089327   5.974648   3.349126   5.946964   
4  129881747  1.567282e+09   6.083040   5.969065   3.349669   5.941364   

   accel_2_y  accel_2_z  accel_3_x  accel_3_y  ...  accel_6_x  accel_6_y  \
0   5.909606   3.361601   6.090122   6.039665  ...   5.890344   5.857710   
1   5.915777   3.361630   6.087561   6.043735  ...   5.886479   5.850656   
2   5.912450   3.359635   6.083359   6.034921  ...   5.886385   5.853241   
3   5.911304   3.365591   6.087561   6.042810  ...   5.888684   5.844299   
4   5.917823   3.361032   6.090989   6.035963  ...   5.885611   5.853917   

   accel_6_z    incl_1    incl_2    incl_3    incl_4    incl_5    incl_6  \
0   3.326484  6.269716

In [3]:
# Secure tunnel variables
host = 'ec2-3-17-203-3.us-east-2.compute.amazonaws.com'
ssh_username = 'ubuntu'
ss_private_key = 'awskey.pem'

In [4]:
# Database variables
POSTGRES_USER='drum_running'
password='king_crimson'
database='MA_DOT'

In [5]:
# Initiate ssh session with drum_running(AWS)
tunnel = SSHTunnelForwarder(
    (host, 22),
    ssh_username=ssh_username,
    ssh_private_key=ss_private_key,
    remote_bind_address=('localhost', 5432),
    local_bind_address=('localhost', 1111),
)

NameError: name 'SSHTunnelForwarder' is not defined

In [6]:
# Function that will execute the SQL command to the server
def pg_data(start_hour, end_hour, table):
    tunnel.start()
    
    conn = psycopg2.connect(
        database=database,
        user=POSTGRES_USER,
        password=password,
        host=tunnel.local_bind_host,
        port=tunnel.local_bind_port,
    )
    dt = datetime.datetime.today()
    
    cur = conn.cursor()
    
    cur.execute("SELECT eventtime, accel_1_x, accel_1_y, accel_1_z, accel_2_x, accel_2_y, accel_2_z, accel_3_x, accel_3_y, accel_3_z, accel_4_x, accel_4_y, accel_4_z, accel_5_x, accel_5_y, accel_5_z, accel_6_x, accel_6_y, accel_6_z FROM {6} WHERE received BETWEEN '2019-{0}-{1} {4}:00:00' AND '2019-{2}-{3} {5}:59:00' ORDER BY id".format(
        dt.month, 
        dt.day -2, 
        dt.month, 
        dt.day -1,
        start_hour,
        end_hour,
        table
    ))
    
    data = cur.fetchall()
    
    conn.close()
    
    tunnel.stop()
    
    return data


In [7]:
# Function that will format the data from PostgreSQL into an organized data structure
def parsed_inc_plot(result):
    accel1 = [[],[],[]]
    accel2 = [[],[],[]]
    accel3 = [[],[],[]]
    accel4 = [[],[],[]]
    accel5 = [[],[],[]]
    accel6 = [[],[],[]]
    received = []

    for i in range(len(result)):      
        accel1[0].append(result[i][7])
        accel1[1].append(result[i][8])
        accel1[2].append(result[i][9])
        
        accel2[0].append(result[i][10])
        accel2[1].append(result[i][11])
        accel2[2].append(result[i][12])
        
        accel3[0].append(result[i][13])
        accel3[1].append(result[i][14])
        accel3[2].append(result[i][15])
        
        accel4[0].append(result[i][16])
        accel4[1].append(result[i][17])
        accel4[2].append(result[i][18])
        
        accel5[0].append(result[i][19])
        accel5[1].append(result[i][20])
        accel5[2].append(result[i][21])
        
        accel6[0].append(result[i][22])
        accel6[1].append(result[i][23])
        accel6[2].append(result[i][24])
        
        received.append(result[i][0])
        
    return { 
        "accel1": accel1,
        "accel2": accel2,
        "accel3": accel3,
        "accel4": accel4,
        "accel5": accel5,
        "accel6": accel6,
        "received": received 
    }



In [10]:
# Function that will format the data from CSV_File into an organized data structure
def parsed_inc_plot(result):
    accel1 = [[],[],[]]
    accel2 = [[],[],[]]
    accel3 = [[],[],[]]
    accel4 = [[],[],[]]
    accel5 = [[],[],[]]
    accel6 = [[],[],[]]
    received = []
      
    accel_1_x = result['accel_1_x'][:]
    accel_1_y = result['accel_1_y'][:]
    accel_1_z = result['accel_1_z'][:]

    accel_2_x = result['accel_2_x'][:]
    accel_2_y = result['accel_2_y'][:]
    accel_2_z = result['accel_2_z'][:]

    accel_3_x = result['accel_3_x'][:]
    accel_3_y = result['accel_3_y'][:]
    accel_3_z = result['accel_3_z'][:]

    accel_4_x = result['accel_4_x'][:]
    accel_4_y = result['accel_4_y'][:]
    accel_4_z = result['accel_4_z'][:]

    accel_5_x = result['accel_5_x'][:]
    accel_5_y = result['accel_5_y'][:]
    accel_5_z = result['accel_5_z'][:]

    accel_6_x = result['accel_6_x'][:]
    accel_6_y = result['accel_6_y'][:]
    accel_6_z = result['accel_6_z'][:]
    
    received = result['received'][:]
    
    for i in range(len(result)):      
        accel1[0].append(accel_1_x[i])
        accel1[1].append(accel_1_y[i])
        accel1[2].append(accel_1_z[i])
        
        accel2[0].append(accel_2_x[i])
        accel2[1].append(accel_2_y[i])
        accel2[2].append(accel_2_z[i])
        
        accel3[0].append(accel_3_x[i])
        accel3[1].append(accel_3_y[i])
        accel3[2].append(accel_3_z[i])
        
        accel4[0].append(accel_4_x[i])
        accel4[1].append(accel_4_y[i])
        accel4[2].append(accel_4_z[i])
        
        accel5[0].append(accel_5_x[i])
        accel5[1].append(accel_5_y[i])
        accel5[2].append(accel_5_z[i])
        
        accel6[0].append(accel_6_x[i])
        accel6[1].append(accel_6_y[i])
        accel6[2].append(accel_6_z[i])
        
        #received.append(received[i])
        
    return { 
        "accel1": accel1,
        "accel2": accel2,
        "accel3": accel3,
        "accel4": accel4,
        "accel5": accel5,
        "accel6": accel6,
        #"received": received 
    }



In [55]:
# Get data from server, parse, format and instantiate linear chart library configuration
plt.rcParams["figure.figsize"] = (50, 10)
result = pg_data("18", "17", "uxbridge_chocolog_st")
parsed_inc_data = parsed_inc_plot(result)
inc = plt
accel = plt

2019-09-30 11:55:17,965| ERROR   | Could not open connection to gateway


BaseSSHTunnelForwarderError: Could not establish session to SSH gateway

In [ ]:
inc.plot(parsed_inc_data['received'], parsed_inc_data['inc1'])
inc.ylabel('Axis - Y')
inc.title("Inclinometer One - Uxbridge, MA Chocolog Street")
inc.xlabel('24 hrs')
inc.grid(True)

inc.show()


In [ ]:
inc.plot(parsed_inc_data['received'], parsed_inc_data['inc2'])
inc.ylabel('Axis - Y')
inc.title("Inclinometer Two - Uxbridge, MA Chocolog Street")
inc.xlabel('24 hrs')
inc.grid(True)

inc.show()


In [ ]:
inc.plot(parsed_inc_data['received'], parsed_inc_data['inc3'])
inc.ylabel('Axis - Y')
inc.title("Inclinometer Three - Uxbridge, MA Chocolog Street")
inc.xlabel('24 hrs')
inc.grid(True)

inc.show()


In [ ]:
inc.plot(parsed_inc_data['received'], parsed_inc_data['inc4'])
inc.ylabel('Axis - Y')
inc.title("Inclinometer Four - Uxbridge, MA Chocolog Street")
inc.xlabel('24 hrs')
inc.grid(True)

inc.show()


In [ ]:
inc.plot(parsed_inc_data['received'], parsed_inc_data['inc5'])
inc.ylabel('Axis - Y')
inc.title("Inclinometer Five - Uxbridge, MA Chocolog Street")
inc.xlabel('24 hrs')
inc.grid(True)

inc.show()


In [ ]:
accel.plot(parsed_inc_data['received'], parsed_inc_data['accel1'][0])
accel.ylabel('Axis - X')
accel.title("Accelerometer One - Uxbridge, MA Chocolog Street")
accel.xlabel('24 hrs')
accel.grid(True)

accel.show()


In [ ]:
accel.plot(parsed_inc_data['received'], parsed_inc_data['accel1'][1])
accel.ylabel('Axis - Y')
accel.title("Accelerometer One - Uxbridge, MA Chocolog Street")
accel.xlabel('24 hrs')
accel.grid(True)

accel.show()


In [ ]:
accel.plot(parsed_inc_data['received'], parsed_inc_data['accel1'][2])
accel.ylabel('Axis - Z')
accel.title("Accelerometer One - Uxbridge, MA Chocolog Street")
accel.xlabel('24 hrs')
accel.grid(True)

accel.show()


In [ ]:
accel.plot(parsed_inc_data['received'], parsed_inc_data['accel2'][0])
accel.ylabel('Axis - X')
accel.title("Accelerometer Two - Uxbridge, MA Chocolog Street")
accel.xlabel('24 hrs')
accel.grid(True)

accel.show()


In [ ]:
accel.plot(parsed_inc_data['received'], parsed_inc_data['accel2'][1])
accel.ylabel('Axis - Y')
accel.title("Accelerometer Two - Uxbridge, MA Chocolog Street")
accel.xlabel('24 hrs')
accel.grid(True)

accel.show()


In [ ]:
accel.plot(parsed_inc_data['received'], parsed_inc_data['accel2'][2])
accel.ylabel('Axis - Z')
accel.title("Accelerometer Two - Uxbridge, MA Chocolog Street")
accel.xlabel('24 hrs')
accel.grid(True)

accel.show()


In [ ]:
accel.plot(parsed_inc_data['received'], parsed_inc_data['accel3'][0])
accel.ylabel('Axis - X')
accel.title("Accelerometer Three - Uxbridge, MA Chocolog Street")
accel.xlabel('24 hrs')
accel.grid(True)

accel.show()


In [ ]:
accel.plot(parsed_inc_data['received'], parsed_inc_data['accel3'][1])
accel.ylabel('Axis - Y')
accel.title("Accelerometer Three - Uxbridge, MA Chocolog Street")
accel.xlabel('24 hrs')
accel.grid(True)

accel.show()


In [ ]:
accel.plot(parsed_inc_data['received'], parsed_inc_data['accel3'][2])
accel.ylabel('Axis - Z')
accel.title("Accelerometer Three - Uxbridge, MA Chocolog Street")
accel.xlabel('24 hrs')
accel.grid(True)

accel.show()


In [ ]:
accel.plot(parsed_inc_data['received'], parsed_inc_data['accel4'][0])
accel.ylabel('Axis - X')
accel.title("Accelerometer Four - Uxbridge, MA Chocolog Street")
accel.xlabel('24 hrs')
accel.grid(True)

accel.show()


In [ ]:
accel.plot(parsed_inc_data['received'], parsed_inc_data['accel4'][1])
accel.ylabel('Axis - Y')
accel.title("Accelerometer Four - Uxbridge, MA Chocolog Street")
accel.xlabel('24 hrs')
accel.grid(True)

accel.show()


In [ ]:
accel.plot(parsed_inc_data['received'], parsed_inc_data['accel4'][2])
accel.ylabel('Axis - Z')
accel.title("Accelerometer Four - Uxbridge, MA Chocolog Street")
accel.xlabel('24 hrs')
accel.grid(True)

accel.show()


In [ ]:
accel.plot(parsed_inc_data['received'], parsed_inc_data['accel5'][0])
accel.ylabel('Axis - X')
accel.title("Accelerometer Five - Uxbridge, MA Chocolog Street")
accel.xlabel('24 hrs')
accel.grid(True)

accel.show()


In [ ]:
accel.plot(parsed_inc_data['received'], parsed_inc_data['accel5'][1])
accel.ylabel('Axis - Y')
accel.title("Accelerometer Five - Uxbridge, MA Chocolog Street")
accel.xlabel('24 hrs')
accel.grid(True)

accel.show()


In [ ]:
accel.plot(parsed_inc_data['received'], parsed_inc_data['accel5'][2])
accel.ylabel('Axis - Z')
accel.title("Accelerometer Five - Uxbridge, MA Chocolog Street")
accel.xlabel('24 hrs')
accel.grid(True)

accel.show()


In [ ]:
accel.plot(parsed_inc_data['received'], parsed_inc_data['accel6'][0])
accel.ylabel('Axis - X')
accel.title("Accelerometer Six - Uxbridge, MA Chocolog Street")
accel.xlabel('24 hrs')
accel.grid(True)

accel.show()


In [ ]:
accel.plot(parsed_inc_data['received'], parsed_inc_data['accel6'][1])
accel.ylabel('Axis - Y')
accel.title("Accelerometer Six - Uxbridge, MA Chocolog Street")
accel.xlabel('24 hrs')
accel.grid(True)

accel.show()


In [ ]:
accel.plot(parsed_inc_data['received'], parsed_inc_data['accel6'][2])
accel.ylabel('Axis - Z')
accel.title("Accelerometer Six - Uxbridge, MA Chocolog Street")
accel.xlabel('24 hrs')
accel.grid(True)

accel.show()
